In [16]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import operator
import random
from sklearn import metrics



In [17]:
dataset = pd.read_csv('train.csv', delimiter=',', header=0)
dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
dataset.isnull().sum(axis = 0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [20]:
dataset['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [21]:
def prepareDataSet(dataset):
    dataset = dataset.fillna({"Embarked": "S"})
    ord_enc = OrdinalEncoder()
    one_hot_enc = OneHotEncoder()
    dataset['sex_code'] = ord_enc.fit_transform(dataset[['Sex']])
    oe_enc = OneHotEncoder()
    oe_results = oe_enc.fit_transform(dataset[["Embarked"]])
    dataset = dataset.join(pd.DataFrame(oe_results.toarray(), columns=oe_enc.categories_))
    return dataset

def getFeaturesValues(dataset):
    return dataset.drop(['PassengerId','Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1)

def getPredictions(dataset):
    return dataset['Survived']

In [22]:
preparedDataSet = prepareDataSet(dataset)
y_df = getPredictions(preparedDataSet)
x_df = getFeaturesValues(preparedDataSet)

In [23]:
preparedDataSet.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex_code,"(C,)","(Q,)","(S,)"
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.0,1.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.0,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1.0,0.0,0.0,1.0


In [24]:
y_df.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [25]:
x_df.head()

,Pclass,Age,SibSp,Parch,Fare,sex_code,"(C,)","(Q,)","(S,)"
0,3,22.0,1,0,7.2500,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,1.0,0.0,0.0,1.0


In [26]:
x_train, x_test, y_train, y_test = train_test_split(x_df.values, y_df.values, test_size=0.15)

In [27]:
def entropy(y):
    entropy = 0
    elemNum = len(y)
    if (elemNum == 0):
        return 0
    ctrDict = getLabelProbaDict(y)
    for label in set(y):
        labelProba = ctrDict[label]
        entropy -=  labelProba * math.log(labelProba)
    return entropy

def getLabelProbaDict(y):
    labels = set(y)
    elemNum = len(y)
    ctrDict = dict(zip(labels, np.zeros(len(labels))))
    for i in range(len(y)):
        ctrDict[y[i]] += 1 / elemNum
    return ctrDict

def getMaxProbaLabel(y):
    return max(getLabelProbaDict(y).items(), key=operator.itemgetter(1))[0]

def getConfidence(y):
    return max(getLabelProbaDict(y).values())

def bootstrap(x):
    return np.random.randint(x, size = x)
    

In [28]:
class TreeNode:
    threshold = float("Nan")
    feature_index = float("Nan")
    
    left_child = None
    right_child = None
    
    class_label = float("Nan")
    confidence = float("Nan")
    
    left_indicies = []
    right_indicies = []
    
    elements = 0
    
    features = []
    
    depth = float("Nan")
    max_tree_depth = float("Nan")
    
    leaf_number = float("Nan")
    max_leaf_number = float("Nan")
    
    max_elemens_per_node = float("Nan")
    
    is_leaf = False
    
    def __init__(self, x, y, tree_depth=1, max_tree_depth=float("Nan"), leaf_number=1, max_leaf_number=float("Nan"), max_elemens_per_node=float("Nan"), features=[]):
        self.elements = len(y)
        
        
        if (len(features) == 0):
            self.features = np.arange(x.shape[1])
        else:
            self.features = features
            
        if (x.shape[0] != len(y)):
            print("ERROR!! WRONG SHAPES")
            
        
        if (len(set(y)) == 1):
            self.is_leaf = True
        
        if (not math.isnan(max_tree_depth)):
            self.depth = tree_depth + 1
            self.max_tree_depth = max_tree_depth
            if (self.depth >= max_tree_depth):
                self.is_leaf = True
        elif (not math.isnan(max_leaf_number)):
            self.leaf_number = leaf_number * 2
            self.max_leaf_number = max_leaf_number
            if (self.leaf_number > max_leaf_number or self.is_leaf):
                self.is_leaf = True
        elif (not math.isnan(max_elemens_per_node)):
            self.max_elemens_per_node = max_elemens_per_node
            if (self.elements <= max_elemens_per_node):
                self.is_leaf = True
        
        if (self.is_leaf):
            self.class_label = getMaxProbaLabel(y)
            self.confidence = getConfidence(y)
        else:
            self.build(x,y)
    
    def predict(self, x_test, withConfidence = False):
        res = []
        for i in range(x_test.shape[0]):
            res.append(self.predictOne(x_test[i], withConfidence))
        return np.array(res)
    
    
    def predictOneDictValue(self, x):
        if (self.is_leaf):
            return {self.class_label:self.confidence}
        elif (math.isnan(x[self.feature_index])):
            return self.left_child.predictOne(x)
        elif (x[self.feature_index] < self.threshold and self.left_child != None):
            return self.left_child.predictOne(x)
        elif (x[self.feature_index] >= self.threshold and self.right_child != None):
            return self.right_child.predictOne(x)
        else:
            print("ERROR! x = " , x," self.left_child = ", self.left_child, " self.right_child = ", self.right_child, " self.is_leaf = ", self.is_leaf)
            return -1
    
    def predictOne(self, x, withConfidence):
        label = -1
        confidence = 0
        if (self.is_leaf):
            label = self.class_label
            confidence = self.confidence
        elif (math.isnan(x[self.feature_index])):
            left_label, left_confidence = self.left_child.predictOne(x, True)
            right_label, right_confidence = self.right_child.predictOne(x, True)
            if (left_confidence > right_confidence):
                label = left_label
                confidence = left_confidence
            else:
                label = right_label
                confidence = right_confidence
        elif (x[self.feature_index] < self.threshold and self.left_child != None):
            if (withConfidence):
                label, confidence = self.left_child.predictOne(x, withConfidence)
            else:
                label = self.left_child.predictOne(x, withConfidence)
        elif (x[self.feature_index] >= self.threshold and self.right_child != None):
            if (withConfidence):
                label, confidence = self.right_child.predictOne(x, withConfidence)
            else:
                label = self.right_child.predictOne(x, withConfidence)
        else:
            print("ERROR! x = " , x," self.left_child = ", self.left_child, " self.right_child = ", self.right_child, " self.is_leaf = ", self.is_leaf)
        if (not withConfidence):
            return label
        else:
            return label, confidence
        
    def build(self, x, y):
        min_G = 2
        for feature in self.features:
            for border in set(x[:,feature]):
                if (math.isnan(border)):
                    continue
                G, left, right = self.getG(x, y, feature, border)
                if (G < min_G):
                    min_G = G
                    self.feature_index = feature
                    self.threshold = border
                    self.left_indicies = left
                    self.right_indicies = right
        if (min_G == 1):
            self.is_leaf = True
            self.class_label = getMaxProbaLabel(y)
            self.confidence = getConfidence(y)
        else:
            if (len(self.left_indicies) > 0):
                self.left_child = TreeNode(x[self.left_indicies], y[self.left_indicies], self.depth, self.max_tree_depth, self.leaf_number, self.max_leaf_number, self.max_elemens_per_node, self.features)
            if (len(self.right_indicies) > 0):
                self.right_child = TreeNode(x[self.right_indicies], y[self.right_indicies], self.depth, self.max_tree_depth, self.leaf_number, self.max_leaf_number, self.max_elemens_per_node, self.features)
            
                
    def getG(self, x, y, feature_index, border):
        left_indicies = []
        right_indicies = []
        for i in range(x.shape[0]):
            if (math.isnan(x[i][feature_index])):
                left_indicies.append(i)
                right_indicies.append(i)
            elif (x[i][feature_index] < border):
                left_indicies.append(i)
            else:
                right_indicies.append(i)
        left_power = len(left_indicies)
        right_power = len(right_indicies)
        total_power = left_power + right_power
        G = left_power / total_power * entropy(y[left_indicies]) + right_power / total_power * entropy(y[right_indicies])
        if (left_power >= len(y) or right_power >= len(y)):
            G = 1
        return G, left_indicies, right_indicies
    

In [32]:
decisionTree = TreeNode(x_train, y_train)


In [33]:
res = decisionTree.predict(x_test)
print(metrics.accuracy_score(y_test, res))

0.8432835820895522


accurcy score for training set

In [34]:
print(metrics.accuracy_score(decisionTree.predict(x_train), y_train))

0.9656538969616909


### try to replace nan age with mean

In [35]:
def replaceNanWithMean(x):
    col_mean = np.nanmean(x, axis = 0)
    inds = np.where(np.isnan(x))
    res = x.copy()
    res[inds] = np.take(col_mean, inds[1])
    return res

In [36]:
decisionTree = TreeNode(replaceNanWithMean(x_train), y_train, max_tree_depth = 50)

In [37]:
res = decisionTree.predict(replaceNanWithMean(x_test))
print(metrics.accuracy_score(y_test, res))

0.8059701492537313


As wee see, accuracy decreased

## DecisionTree from sklearn

In [38]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth=50)
clf.fit(replaceNanWithMean(x_train), y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=50, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [39]:
res = clf.predict(replaceNanWithMean(x_test))
print(metrics.accuracy_score(y_test, res))

0.7313432835820896


not so good

### try to exclude last 3 features with my TreeNode class

In [40]:
decisionTree = TreeNode(x_train[:, :-3], y_train)

In [41]:
res = decisionTree.predict(x_test)
print(metrics.accuracy_score(y_test, res))

0.8283582089552238


As we see, accuracy decreased, so the last 3 features have some information

### let's try the same, but using mean age instead of NaN values

In [42]:
decisionTree = TreeNode(replaceNanWithMean(x_train[:, :-3]), y_train)

In [43]:
res = decisionTree.predict(x_test)
print(metrics.accuracy_score(y_test, res))

0.7835820895522388


So, the current strategy with placing elements with NaN values in both left and right nodes works better, than inserting mean values instead of NaNs

## Bagging

In [114]:
def buildBaggingGroup(x_train, y_train, number):
    trees = []
    for i in range(number):
        indicies = bootstrap(len(y_train))
        trees.append(TreeNode(x_train[indicies], y_train[indicies]))
    return trees
        
def ansemblePredict(trees, x_test):
    predictions = []
    for tree in trees:
        predictions.append(tree.predict(x_test))
    return np.rint(np.mean(np.array(predictions), axis = 0))


def ansemblePredictWithConfidence(trees, x_test):
    prediction_labels = []
    prediction_confidences = []
    for tree in trees:
        one_prediction = tree.predict(x_test, True)
        prediction_labels.append(one_prediction[:,0])
        prediction_confidences.append(one_prediction[:,1])
    result = []
    prediction_confidences = np.array(prediction_confidences).reshape(x_test.shape[0], len(trees))
    prediction_labels = np.array(prediction_labels).reshape(x_test.shape[0], len(trees))
    for i in range(prediction_confidences.shape[0]):
        if (max(prediction_confidences[i]) >= 1.0):
            total = dict()
            for j in range(prediction_confidences.shape[1]):
                if (prediction_confidences[i][j] >= 1.0):
                    if (total.get(prediction_labels[i][j]) == None):
                        total[prediction_labels[i][j]] = 0
                    total[prediction_labels[i][j]] += 1
            result.append(np.rint(max(total.items(), key=operator.itemgetter(1))[0]))
        else:
            result.append(prediction_labels[i][np.argmax(prediction_confidences[i])])
    return np.array(result)

In [99]:
a = {1:4}
print(a.get(3))
a[4] = -1
a

None


{1: 4, 4: -1}

In [48]:
bagging_forest = buildBaggingGroup(x_train, y_train, 70)

In [116]:
res = ansemblePredict(bagging_forest, x_test)
print(metrics.accuracy_score(y_test, res))

0.8731343283582089


#### as we see, accuracy increased on 3 %, now will try the same bagging forest, but decision will be made according to the max confidence

## Random Forest

In [118]:
def buildRandomForest(x_train, y_train, number, max_features):
    trees = []
    for i in range(number):
        indicies = bootstrap(len(y_train))
        trees.append(TreeNode(x_train[indicies], y_train[indicies], features = random.sample(list(np.arange(x_train.shape[1])), max_features)))
    return trees

In [119]:
random_forest = buildRandomForest(x_train, y_train, 70, x_train.shape[1] - 2)

In [120]:
res = ansemblePredict(random_forest, x_test)
print(metrics.accuracy_score(y_test, res))

0.8656716417910447


## Random Forest from sklearn

In [123]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=70, criterion = 'entropy', random_state=42)
rf_clf.fit(replaceNanWithMean(x_train), y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=70,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [124]:
res_rf = rf_clf.predict(replaceNanWithMean(x_test))
print(metrics.accuracy_score(y_test, res_rf))

0.7985074626865671


much better than for simple decision tree from sklearn, but still worse than my forest (because of NaNs)

In [125]:
from sklearn.ensemble import RandomForestClassifier
rf_clf_gini = RandomForestClassifier(n_estimators=70, criterion = 'gini', random_state=42)
rf_clf_gini.fit(replaceNanWithMean(x_train), y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=70,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [126]:
res_rf_gini = rf_clf_gini.predict(replaceNanWithMean(x_test))
print(metrics.accuracy_score(y_test, res_rf_gini))

0.8059701492537313


As we see, Gini criterion gives better result